<a href="https://colab.research.google.com/github/ZeynepHeray/Algo_trading/blob/master/DeepFake_DetectionV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install face_recognition

In [ ]:
import json
import cv2
import os
import torch
import face_recognition
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report
import copy

In [ ]:
with open('/content/drive/MyDrive/trainData/metadata.json') as f:
    labels = json.load(f)

In [ ]:
def extract_frames_and_labels(video_path, label, frame_interval=30):
    video_capture = cv2.VideoCapture(video_path)
    frames = []
    labels = []
    frame_count = 0

    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            face_locations = face_recognition.face_locations(frame)
            for face_location in face_locations:
                top, right, bottom, left = face_location
                face_image = frame[top:bottom, left:right]
                face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
                face_image = Image.fromarray(face_image)
                frames.append(face_image)
                labels.append(label)
        frame_count += 1

    video_capture.release()
    return frames, labels


In [ ]:
import pickle

# Frame'leri ve etiketleri kaydetmek için dizin oluşturma
frames_dir = '/content/drive/MyDrive/trainData/frames'
os.makedirs(frames_dir, exist_ok=True)

all_frames = []
all_labels = []

for video_file, info in labels.items():
    label = 0 if info['label'] == 'REAL' else 1
    video_path = os.path.join('/content/drive/MyDrive/trainData/dfdc_train_part_32', video_file)
    frames, frame_labels = extract_frames_and_labels(video_path, label)
    all_frames.extend(frames)
    all_labels.extend(frame_labels)

# Frame'leri ve etiketleri pickle dosyasına kaydetme
with open(os.path.join(frames_dir, 'all_frames.pkl'), 'wb') as f:
    pickle.dump(all_frames, f)
with open(os.path.join(frames_dir, 'all_labels.pkl'), 'wb') as f:
    pickle.dump(all_labels, f)
